In [1]:
import nipype

from nipype.interfaces import fsl
from nipype.interfaces import ants
from nipype.interfaces import freesurfer
from os.path import abspath

In [2]:
base_dir=abspath('examples/')

In [3]:
CORES=4
TASKS=2

In [4]:
grabber = nipype.Node(interface=nipype.DataGrabber(infields=['arg'],outfields=['out_file']), name='grabber')      
grabber.inputs.base_directory = base_dir
grabber.inputs.sort_filelist = False
grabber.inputs.template = '*/%s.nii.gz'
grabber.inputs.arg = 't1w'

In [5]:
#a=grabber.run()
#a.outputs

In [6]:
sink = nipype.Node(interface=nipype.DataSink(),name='sink')
sink.inputs.base_directory = base_dir
# this is to force sink to use the inputs dirs 
# credits: https://gist.github.com/lebedov/9294b8b37640db911bfc987aca49f239
#sink.inputs.regexp_substitutions = [('_\w+\d+', '')]

In [7]:
# Neck removal by FSL robustfov
neck_remove=nipype.MapNode(interface=fsl.RobustFOV(), name='neck_remove', iterfield=['in_file'])
neck_remove.inputs.out_roi="t1w_fov.nii.gz"

In [37]:
# Field Inhomogenity estimation (if any) and removal by ANTs N4BiasFieldCorrection
N4_FC=nipype.MapNode(interface=ants.N4BiasFieldCorrection(), name="N4_FC", iterfield=['input_image'])
N4_FC.inputs.dimension = 3
N4_FC.inputs.output_image="t1w_fov_N4.nii.gz"
N4_FC.inputs.num_threads=CORES

In [10]:
workflow = nipype.Workflow('workflow')
#workflow.connect([(grabber, neck_remove, [('out_file', 'in_file')]),
#                  (neck_remove, N4_FC, [('out_roi', 'input_image')]),
#                  (N4_FC, sink, [('output_image', 'preproc.@in_file')]),
#                 ])
workflow.connect([(grabber, neck_remove, [('out_file', 'in_file')]),
                  (neck_remove, sink, [('out_roi', '@in_file')]),
                 ])

In [11]:
#workflow.run()
workflow.run('MultiProc', plugin_args={'n_procs': TASKS})

170921-17:55:09,979 workflow INFO:
	 Workflow workflow settings: ['check', 'execution', 'logging']
170921-17:55:10,22 workflow INFO:
	 Running in parallel.
170921-17:55:10,26 workflow INFO:
	 Executing: grabber ID: 0
170921-17:55:10,32 workflow INFO:
	 Executing node grabber in dir: /var/folders/5_/33p0sl3j2_ngqn_gghyy2zj00000gn/T/tmpF7obBI/workflow/grabber
170921-17:55:10,99 workflow INFO:
	 [Job finished] jobname: grabber jobid: 0
170921-17:55:10,102 workflow INFO:
	 Executing: neck_remove ID: 1
170921-17:55:10,113 workflow INFO:
	 Adding 2 jobs for mapnode neck_remove
170921-17:55:10,117 workflow INFO:
	 Executing: _neck_remove0 ID: 3
170921-17:55:10,120 workflow INFO:
	 Executing: _neck_remove1 ID: 4
170921-17:55:10,125 workflow INFO:
	 Executing node _neck_remove1 in dir: /var/folders/5_/33p0sl3j2_ngqn_gghyy2zj00000gn/T/tmpOX9MdR/workflow/neck_remove/mapflow/_neck_remove1
170921-17:55:10,124 workflow INFO:
	 Executing node _neck_remove0 in dir: /var/folders/5_/33p0sl3j2_ngqn_gghyy